In [9]:
!pip install -r requirements.txt

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.4/165.4 kB 961.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 27.0 MB/s eta 0:00:0000:010:01
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 51.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 50.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 27.6 MB/s eta 0:00:00


In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models , initializers
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os


# load data

In [ ]:
def load_images_from_dir(directory, target_size=(320, 320)):
    image_list = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(directory, filename)
            # Load the image using OpenCV
            image = cv2.imread(image_path)
            if image is None:
                print(f"Warning: Unable to load image '{filename}'")
                continue
            
            # Convert BGR to RGB format
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Resize the image
            resized_image = cv2.resize(image_rgb, target_size)
            image_list.append(resized_image)
    return image_list



In [ ]:
directory_path = "data/monet_jpg/"
monet_image_list = load_images_from_dir(directory_path)
print("Number of images loaded:", len(monet_image_list))

In [ ]:
directory_path = "data/photo_jpg/"
image_list = load_images_from_dir(directory_path)
print("Number of images loaded:", len(image_list))

# EDA images

In [ ]:
def plot_random_images(image_list):
    # Ensure that we have at least 5 images
    if len(image_list) < 5:
        print("Error: Insufficient number of images.")
        return
    
    # Choose 5 random indices
    random_indices = random.sample(range(len(image_list)), 5)
    
    # Plot the images
    fig, axes = plt.subplots(1, 5, figsize=(15, 5))
    for i, idx in enumerate(random_indices):
        axes[i].imshow(image_list[idx])  # Convert BGR to RGB for Matplotlib
        axes[i].axis('off')
        axes[i].set_title(f"Image {idx}")
    plt.show()


In [ ]:
plot_random_images(image_list)

In [ ]:
plot_random_images(monet_image_list)

# cleaning the data

In [ ]:
def has_white_corner(image):
    # Convert image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    # Define the size of the corner region
    corner_size = 5
    
    # Define the four corners
    corners = [
        gray_image[:corner_size, :corner_size],                # Top-left corner
        gray_image[:corner_size, -corner_size:],              # Top-right corner
        gray_image[-corner_size:, :corner_size],              # Bottom-left corner
        gray_image[-corner_size:, -corner_size:]              # Bottom-right corner
    ]
    
    # Check if all corners are completely white
    return all(np.all(corner > 220) for corner in corners)

def crop_and_resize(image, target_size=(320, 320)):
    # Get the dimensions of the input image
    height, width = image.shape[:2]
    
    # Calculate the center coordinates
    center_x = width // 2
    center_y = height // 2
    
    # Calculate the crop boundaries
    crop_left = max(0, center_x - 215 // 2)
    crop_top = max(0, center_y - 215 // 2)
    crop_right = min(width, center_x + 215 // 2)
    crop_bottom = min(height, center_y + 215 // 2)
    
    # Crop the image
    cropped_image = image[crop_top:crop_bottom, crop_left:crop_right]
    
    # Resize the cropped image
    resized_image = cv2.resize(cropped_image, target_size)
    
    return resized_image

def show_images_with_white_corner(images):
    for idx, image in enumerate(images):
        if has_white_corner(image):
            cropped_resized_image = crop_and_resize(image)
            plt.imshow(image)
            plt.title(f"Image {idx}")
            plt.axis('off')
            plt.show()

            plt.imshow(cropped_resized_image)
            plt.title(f"Image {idx}")
            plt.axis('off')
            plt.show()


show_images_with_white_corner(monet_image_list)

In [ ]:
def change_images_with_white_corner(images):
    new_images = []
    for idx, image in enumerate(images):
        if has_white_corner(image):
            new_images.append(crop_and_resize(image))
        else:
            new_images.append(image)

    return new_images

monet_image_list = change_images_with_white_corner(monet_image_list)

# AUGMENTATION

In [ ]:
def split_and_resize_quarters(original_image):
    # Convert PIL Image to NumPy array
    original_image_array = np.array(original_image)
    
    # Ensure the image is 320x320
    if original_image_array.shape[:2] != (320, 320):
        print("Error: Image size must be 320x320.")
        return None
    
    
    # Split the image into quarters
    quarters = []
    for i in range(2):
        for j in range(2):
            left = j * 160
            upper = i * 160
            right = left + 160
            lower = upper + 160
            quarter = original_image_array[upper:lower, left:right, :]
            quarters.append(quarter)
    
    # Resize each quarter to 320x320
    resized_quarters = [cv2.resize(quarter, (320, 320)) for quarter in quarters]
    
    return original_image , resized_quarters

In [ ]:
original_image, quarters = split_and_resize_quarters(monet_image_list[0])
if quarters:
    # Plot original image
    plt.figure(figsize=(8, 8))
    plt.imshow(original_image)
    plt.axis('off')
    plt.title("Original Image")
    plt.show()
    
    # Plot each quarter
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))
    for i, quarter in enumerate(quarters):
        axes[i//2, i%2].imshow(quarter)
        axes[i//2, i%2].axis('off')
        axes[i//2, i%2].set_title(f"Quarter {i+1}")
    plt.show()

In [ ]:
def get_images_with_high_color_variance(images, threshold):
    high_color_var_images = []
    
    for image in images:
        # Convert image to numpy array
        image_array = np.array(image)
        
        # Calculate color variance
        color_variance = np.var(image_array)
        # Check if color variance is higher than the threshold
        if color_variance > threshold:
            high_color_var_images.append(image)
    
    return high_color_var_images

In [ ]:
most_colorful_image = get_images_with_high_color_variance(quarters , 1000)
if most_colorful_image:
    # Plot each quarter
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))
    for i, quarter in enumerate(most_colorful_image):
        axes[i//2, i%2].imshow(quarter)
        axes[i//2, i%2].axis('off')
    plt.show()

In [ ]:
def mirror_flip(image):
    # Mirror flip the image horizontally
    flipped_image = cv2.flip(image, 1)
    return flipped_image


def aug_pipeline(images, threshold):
    augmented_images = []
    
    for image in images:
        # Split into quarters
        _ , quarters = split_and_resize_quarters(image)
        
        # Save only important quarters
        important_quarters = get_images_with_high_color_variance(quarters, threshold)

        # Mirror flip the original image
        mirrored_image = mirror_flip(image)
                
        # Combine original image, mirrored image, and important quarters
        augmented_images.append(image)
        augmented_images.append(mirrored_image)
        if len(important_quarters) != 0:
            augmented_images.extend(important_quarters)
    
    return augmented_images

In [ ]:
print(len(monet_image_list))
augmented_images = aug_pipeline(monet_image_list, 1000)
print(len(augmented_images))
plot_random_images(augmented_images)

# save the data in batch for model

In [ ]:
# Define batch size
batch_size = 10

# Convert lists to TensorFlow datasets
monet_dataset = tf.data.Dataset.from_tensor_slices(augmented_images)
photo_dataset = tf.data.Dataset.from_tensor_slices(image_list)

# Zip the datasets to combine them
combined_dataset = tf.data.Dataset.zip((monet_dataset, photo_dataset))

# Optionally shuffle and batch the dataset
combined_dataset = combined_dataset.shuffle(buffer_size=len(augmented_images)).batch(batch_size)



In [ ]:
import matplotlib.pyplot as plt

def plot_images_from_batch(dataset, title):
    # Get one batch of images from the dataset
    images_batch = next(iter(dataset.batch(10)))  # Assuming batch size of 16
    # Create a figure with a larger size
    plt.figure(figsize=(12, 8))
    # Plot each image in the batch
    for i, image in enumerate(images_batch):
        plt.subplot(4, 4, i + 1)
        plt.imshow(image.numpy().astype("uint8"))  # Convert to uint8 for plotting
        plt.axis("off")
    # Set title and show plot
    plt.suptitle(title)
    plt.show()

# Plot one batch of Monet images
plot_images_from_batch(monet_dataset, "Monet Images")

# Plot one batch of photo images
plot_images_from_batch(photo_dataset, "Photo Images")


# create the objects that neccessary for the training

In [ ]:

def downsample(filters, size, apply_batchnorm=True):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))

  if apply_batchnorm:
    result.add(tf.keras.layers.BatchNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result


sample = next(iter(monet_dataset.batch(1)))  # Assuming batch size of 1
sample_float32 = tf.cast(sample, tf.float32)

down_model = downsample(3, 4)
down_result = down_model(sample_float32)
print(down_result.shape)

In [ ]:

def upsample(filters, size, apply_dropout=False):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

  result.add(tf.keras.layers.BatchNormalization())

  if apply_dropout:
      result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())

  return result
     


up_model = upsample(3, 4)
up_result = up_model(down_result)
print (up_result.shape)

In [ ]:


# Define Generator model
def Generator():
  inputs = tf.keras.layers.Input(shape=[320, 320, 3])
  
  down_stack = [
    downsample(80, 4, apply_batchnorm=False),  # (batch_size, 160, 160, 80)
    downsample(160, 4),  # (batch_size, 64, 64, 128)
    downsample(320, 4),  # (batch_size, 32, 32, 256)
    downsample(640, 4),  # (batch_size, 16, 16, 512)
    downsample(640, 4),  # (batch_size, 8, 8, 512)
    downsample(640, 4),  # (batch_size, 4, 4, 512)
    # downsample(640, 4),  # (batch_size, 2, 2, 512)
    # downsample(640, 4),  # (batch_size, 1, 1, 512)
  ]

  up_stack = [
    # upsample(640, 4, apply_dropout=True),  # (batch_size, 2, 2, 1024)
    # upsample(640, 4, apply_dropout=True),  # (batch_size, 4, 4, 1024)
    upsample(640, 4, apply_dropout=True),  # (batch_size, 8, 8, 1024)
    upsample(640, 4 ,apply_dropout=True),  # (batch_size, 16, 16, 1024)
    upsample(320, 4),  # (batch_size, 32, 32, 512)
    upsample(160, 4),  # (batch_size, 64, 64, 256)
    upsample(80, 4),  # (batch_size, 128, 128, 128)
  ]

  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(3, 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh')  # (batch_size, 256, 256, 3)



  x = inputs


  # Downsampling through the model
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = tf.keras.layers.Concatenate()([x, skip])

  x = last(x)
     


  return tf.keras.Model(inputs=inputs, outputs=x)

# Test the generator model with random input batch
def test_build_generator():


    # Build generator model
    generator = Generator()
    

    input_shape = (320, 320, 3)  # Example input shape
    # Generate a random batch of input images
    batch_size = 4
    random_input_batch = tf.random.uniform((batch_size,) + input_shape, minval=0, maxval=1)
    
    # Forward pass through the generator
    generated_images = generator(random_input_batch)
    print(generator.summary())

    # Check output shape
    output_shape = generated_images.shape
    expected_output_shape = (batch_size, 320, 320, 3)  # Output shape with batch dimension
    
    # Check if the output shape matches the expected output shape
    assert output_shape == expected_output_shape, "Output shape mismatch"
    print("Test passed!")

# Run the test
test_build_generator()

In [ ]:
# Define Discriminator model
def Discriminator(input_nc = 3, ndf=64, n_layers=3):
    inputs = tf.keras.layers.Input(shape=(None, None, input_nc))
    x = inputs
    nf_mult_prev = 1

    # First convolutional layer
    x = tf.keras.layers.Conv2D(ndf, kernel_size=4, strides=2, padding='same')(x)
    x = tf.keras.layers.LeakyReLU(0.2)(x)

    nf_mult = 1
    # Middle convolutional layers
    for n in range(1, n_layers):
        nf_mult_prev = nf_mult
        nf_mult = min(2 ** n, 8)
        x = tf.keras.layers.Conv2D(ndf * nf_mult, kernel_size=4, strides=2, padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(0.2)(x)

    # Last convolutional layer
    nf_mult_prev = nf_mult
    nf_mult = min(2 ** n_layers, 8)
    x = tf.keras.layers.Conv2D(ndf * nf_mult, kernel_size=4, strides=1, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(0.2)(x)

    # Output convolutional layer
    outputs = tf.keras.layers.Conv2D(1, kernel_size=4, strides=1, padding='same')(x)

    return tf.keras.Model(inputs=inputs, outputs=outputs)

# Test the discriminator model with random input batch
def test_build_discriminator():
    # Define input shape
    input_shape = (320, 320, 3)  # Example input shape

    # Build discriminator model
    discriminator = Discriminator(3)
    
    # Generate a random batch of input images
    batch_size = 4
    random_input_batch = tf.random.uniform((batch_size,) + input_shape, minval=0, maxval=1)
    
    # Forward pass through the discriminator
    discriminator_output = discriminator(random_input_batch)
    
    # Check output shape
    output_shape = discriminator_output.shape
    print(output_shape)
    print(discriminator.summary())
    expected_output_shape = (batch_size, 40, 40, 1)  # Output shape with batch dimension
    
    # Check if the output shape matches the expected output shape
    assert output_shape == expected_output_shape, "Output shape mismatch"
    print("Test passed!")

# Run the test
test_build_discriminator()

In [ ]:
# Adversarial Loss
def adversarial_loss(discriminator, generated):
    fake_output = discriminator(generated)
    return tf.reduce_mean(tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(fake_output), fake_output))


# Cycle Consistency Loss
def cycle_consistency_loss(real_images, cycled_images, lambda_weight=10):
    real_images = tf.cast(real_images, tf.float32)
    cycled_images = tf.cast(cycled_images, tf.float32)
    loss = tf.reduce_mean(tf.abs(real_images - cycled_images))
    return lambda_weight * loss

# Identity Loss
def identity_loss(real_images, same_images, lambda_weight=0.5):
    real_images = tf.cast(real_images, tf.float32)
    same_images = tf.cast(same_images, tf.float32)
    loss = tf.reduce_mean(tf.abs(real_images - same_images))
    return lambda_weight * 0.5 * loss



def discriminator_loss(real_output, fake_output):
    real_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(real_output), real_output)
    fake_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss


In [ ]:
def train_one_batch(generator_real2monet, generator_monet2real, discriminator_real, discriminator_monet, batch_real, batch_monet):
    with tf.GradientTape() as gen_R2M_tape, tf.GradientTape() as gen_M2R_tape, \
         tf.GradientTape() as disc_r_tape, tf.GradientTape() as disc_m_tape:

        # Generate fake images
        fake_monet = generator_real2monet(batch_real, training=True)
        fake_real = generator_monet2real(batch_monet, training=True)

        # Generate reconstructed images
        reconstr_real = generator_monet2real(fake_monet, training=True)
        reconstr_monet = generator_real2monet(fake_real, training=True)

        # Identity mapping loss
        identity_loss_real = identity_loss(batch_real, reconstr_real)
        identity_loss_monet = identity_loss(batch_monet, reconstr_monet)

        # Adversarial loss
        adv_loss_R2M = adversarial_loss(discriminator_monet, fake_monet)
        adv_loss_M2R = adversarial_loss(discriminator_real, fake_real)


        # Cycle consistency loss
        cycle_loss = cycle_consistency_loss(batch_real, reconstr_real) + \
                     cycle_consistency_loss(batch_monet, reconstr_monet)

        # Total generator loss
        total_gen_R2M_loss = adv_loss_R2M + cycle_loss
        total_gen_M2R_loss = adv_loss_M2R + cycle_loss

    # Compute gradients of generator loss with respect to generator variables
    gen_R2M_gradients = gen_R2M_tape.gradient(total_gen_R2M_loss, generator_real2monet.trainable_variables)
    gen_M2R_gradients = gen_M2R_tape.gradient(total_gen_M2R_loss, generator_monet2real.trainable_variables)

    # Apply gradients to generator variables
    generator_real2monet.optimizer.apply_gradients(zip(gen_R2M_gradients, generator_real2monet.trainable_variables))
    generator_monet2real.optimizer.apply_gradients(zip(gen_M2R_gradients, generator_monet2real.trainable_variables))

    # Train discriminator X
    with tf.GradientTape() as disc_r_tape:
        disc_real_real_output = discriminator_real(batch_real, training=True)
        disc_real_fake_output = discriminator_real(fake_real, training=True)

        # Compute discriminator X loss
        disc_real_loss = discriminator_loss(disc_real_real_output, disc_real_fake_output)

    # Compute gradients of discriminator X loss with respect to discriminator X variables
    disc_real_gradients = disc_r_tape.gradient(disc_real_loss, discriminator_real.trainable_variables)

    # Apply gradients to discriminator X variables
    discriminator_real.optimizer.apply_gradients(zip(disc_real_gradients, discriminator_real.trainable_variables))

    # Train discriminator Y
    with tf.GradientTape() as disc_m_tape:
        disc_monet_real_output = discriminator_monet(batch_monet, training=True)
        disc_monet_fake_output = discriminator_monet(fake_monet, training=True)

        # Compute discriminator Y loss
        disc_monet_loss = discriminator_loss(disc_monet_real_output, disc_monet_fake_output)

    # Compute gradients of discriminator Y loss with respect to discriminator Y variables
    disc_monet_gradients = disc_m_tape.gradient(disc_monet_loss, discriminator_monet.trainable_variables)

    # Apply gradients to discriminator Y variables
    discriminator_monet.optimizer.apply_gradients(zip(disc_monet_gradients, discriminator_monet.trainable_variables))




In [ ]:
def train(generator_real2monet, generator_monet2real, discriminator_real, discriminator_monet, combined_dataset, epochs):
    num_batches = len(combined_dataset)
    
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        
        # Use tqdm to create a progress bar for the batches
        progress_bar = tqdm(combined_dataset, total=num_batches, desc=f"Epoch {epoch + 1}/{epochs}")
        
        for batch_real, batch_monet in progress_bar:
            train_one_batch(generator_real2monet, generator_monet2real, discriminator_real, discriminator_monet, batch_real, batch_monet)
        



In [ ]:

# Build and compile the generators
generator_real2monet = Generator()
generator_monet2real = Generator()
generator_real2monet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), loss=adversarial_loss)
generator_monet2real.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), loss=adversarial_loss)

# Build and compile the discriminators
discriminator_real = Discriminator()
discriminator_monet = Discriminator()
discriminator_real.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), loss=adversarial_loss)
discriminator_monet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), loss=adversarial_loss)

In [ ]:
train(generator_real2monet, generator_monet2real, discriminator_real, discriminator_monet, combined_dataset, epochs=10)
